# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [43]:
# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

# import libraries
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re
import pickle

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amira\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amira\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amira\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_messages.db')
df = pd.read_sql_table('messages', engine)
df.head() 

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# feature & target values X # y
X = df.message.values 
y = df.iloc[:, 4:]
y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [20]:
def tokenize(text):
    # Remove urls
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_regex, 'urlplaceholder', text)

    # Expand contractions before we remove punctuation, is it necessary in this context?
    # https://medium.com/@lukei_3514/dealing-with-contractions-in-nlp-d6174300876b
    
    # Convert to lower case and remove punctuation
    text = re.sub(r'[^a-zA-Z0-9]', ' ' , text.lower()) 

    # TODO: now - is removed also between numbers, let's see if keeping them affects our model
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    
    # lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]

    return tokens
    

In [21]:
# Test tokenize method with a dummy text
msg = "UN http://un.com reports Leogane 80-90 destroyed. Only Hospital St.   Croix http://www.hospital.org/hos functioning. Needs supplies desperately. We can not survive"
tokenize(msg)

['un',
 'urlplaceholder',
 'report',
 'leogane',
 '80',
 '90',
 'destroyed',
 'hospital',
 'st',
 'croix',
 'urlplaceholder',
 'functioning',
 'need',
 'supply',
 'desperately',
 'survive']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [23]:
# Build pipeline
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=0, n_estimators=200)))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
13763,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
21716,1,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16002,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3745,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12096,1,0,0,1,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6208,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5374,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8729,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [27]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [28]:
def display_results(y_test, y_pred):
    categories = y_test.columns
    print(classification_report(y_test, y_pred, target_names=categories))
 

In [29]:
# display results
display_results(y_test, y_pred)

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      5045
               request       0.83      0.49      0.62      1135
                 offer       0.00      0.00      0.00        23
           aid_related       0.77      0.68      0.72      2742
          medical_help       0.67      0.06      0.12       524
      medical_products       0.80      0.10      0.18       327
     search_and_rescue       0.86      0.03      0.06       190
              security       0.00      0.00      0.00       109
              military       0.84      0.11      0.19       199
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.40      0.55       424
                  food       0.87      0.65      0.74       727
               shelter       0.83      0.40      0.54       588
              clothing       0.73      0.07      0.13       109
                 money       1.00      

C:\Users\amira\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amira\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amira\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amira\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [30]:
def display_results_details(y_test, y_pred):
    categories = y_test.columns
    for i in range(len(categories)):
        category = categories[i] 
        print ('Category: ' + category)
        print(classification_report(y_test[category], y_pred[:, i] , labels=np.unique(y_test[category])))

display_results_details(y_test, y_pred)        

Category: related
              precision    recall  f1-score   support

           0       0.69      0.38      0.49      1509
           1       0.84      0.95      0.89      5045

    accuracy                           0.82      6554
   macro avg       0.77      0.66      0.69      6554
weighted avg       0.80      0.82      0.80      6554

Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5419
           1       0.83      0.49      0.62      1135

    accuracy                           0.90      6554
   macro avg       0.87      0.74      0.78      6554
weighted avg       0.89      0.90      0.88      6554

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6531
           1       0.00      0.00      0.00        23

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.9

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6477
           1       0.00      0.00      0.00        77

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Category: other_infrastructure
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6235
           1       0.20      0.00      0.01       319

    accuracy                           0.95      6554
   macro avg       0.58      0.50      0.49      6554
weighted avg       0.91      0.95      0.93      6554

Category: weather_related
              precision    recall  f1-score   support

           0       0.89      0.96      0.92      4688
           1       0.87      0.69      0.77      1866

    accuracy                           0.88      6554
   macro avg       0.88      0.83      0.85      6554
weighted avg     

### 6. Improve your model
Use grid search to find better parameters. 

In [31]:
# View pipeline parameters 
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001B86D768708>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

In [32]:
# Use GridSearchCV to find better parameters 
parameters = { 'clf__estimator__criterion': ['gini', 'entropy'] }

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=2, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [33]:
# run the Grid Search
cv.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed: 45.1min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [48]:
cv.best_params_

{'clf__estimator__criterion': 'gini'}

In [51]:
cv.best_score_  
# why it is too low? maybe related to the fact that the y is multi-label? 
# so it needs certain adjustments in how score should be calculated?

0.27387843530716305

In [37]:
# predict on test data
y_pred = cv.predict(X_test)

In [42]:
accuracy_score(y_test.values.reshape(-1, 1), y_pred.reshape(-1, 1))

0.9485471128742413

In [ ]:
# display results
display_results(y_test, y_pred)

In [ ]:
# display detailed results
display_results_details(y_test, y_pred)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [47]:
with open('classifier.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.